## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-24-16-46-00 +0000,wsj,Opinion | Uncle Sam Shouldn’t Own Intel Stock,https://www.wsj.com/opinion/uncle-sam-shouldnt...
1,2025-08-24-16-45-00 +0000,wsj,Opinion | The Doctors Who Cry ‘Science’,https://www.wsj.com/opinion/the-doctors-who-cr...
2,2025-08-24-16-41-00 +0000,wsj,President Trump’s attempts to control monetary...,https://www.wsj.com/economy/central-banking/fe...
3,2025-08-24-16-37-35 +0000,nypost,Passenger’s device catches fire on American Ai...,https://nypost.com/2025/08/24/us-news/passenge...
4,2025-08-24-16-34-05 +0000,nypost,Rep. Lawler backs Trump sending National Guard...,https://nypost.com/2025/08/24/us-news/rep-lawl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,22
165,new,11
202,ukraine,8
9,president,7
210,all,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
19,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...,54
140,2025-08-23-17-43-00 +0000,wsj,Chicago will likely be the next target in what...,https://www.wsj.com/politics/policy/trump-look...,53
9,2025-08-24-15-54-42 +0000,nypost,House Dem leader Jeffries accuses Trump of try...,https://nypost.com/2025/08/24/us-news/hakeem-j...,51
76,2025-08-24-07-02-31 +0000,nypost,Illinois Gov. Pritzker says Trump trying to ‘m...,https://nypost.com/2025/08/24/us-news/illinois...,47
60,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...,46


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
19,54,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...
55,42,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
94,34,2025-08-24-02-00-00 +0000,wsj,Bashing the president’s tariff policies could ...,https://www.wsj.com/politics/policy/gop-tariff...
96,31,2025-08-24-01-14-02 +0000,nypost,"Rikers Island inmate found dead in cell, marki...",https://nypost.com/2025/08/23/us-news/rikers-i...
111,28,2025-08-23-21-52-29 +0000,nypost,Israel says it will begin Gaza City military t...,https://nypost.com/2025/08/23/world-news/israe...
125,27,2025-08-23-20-15-36 +0000,nypost,"Putin war hero, the ‘Executioner,’ exposed as ...",https://nypost.com/2025/08/23/world-news/putin...
104,26,2025-08-24-00-05-20 +0000,nypost,Zohran Mamdani fails at bench press while tryi...,https://nypost.com/2025/08/23/us-news/zohran-m...
31,22,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...
121,21,2025-08-23-20-47-34 +0000,nyt,Abrego Garcia’s Lawyers Say U.S. Officials Tri...,https://www.nytimes.com/2025/08/23/us/politics...
84,21,2025-08-24-04-53-54 +0000,nypost,Rhode Island prosecutor who told cops they’d ‘...,https://nypost.com/2025/08/24/us-news/rhode-is...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
